# Check consistency of bias between different measurements.

There are x measurements:
- autocorrelation (noRSD). Lognormal term
- cross-correlation with unclustered (noRSD). cross-term
- cross-correlation with unclustered (RSD). rsd-term + cross-term

In [1]:
source=5
source_alt=4

In [2]:
from CoLoRe_corrf_analysis.plot_methods import Plots
from CoLoRe_corrf_analysis.file_funcs import FileFuncs
from CoLoRe_corrf_analysis.fitter import Fitter
from CoLoRe_corrf_analysis.read_colore import ComputeModelsCoLoRe

from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

In [3]:
theory_path=Path('/global/cscratch1/sd/damonge/CoLoRe_sims/sim1000')
bias_filename=Path('/global/cscratch1/sd/cramirez/NBodyKit/hanyu_david_box/input_files/BzBlue.txt')
nz_filename=Path('/global/cscratch1/sd/cramirez/NBodyKit/hanyu_david_box/input_files/NzBlue.txt')
pk_filename=Path('/global/cscratch1/sd/cramirez/NBodyKit/hanyu_david_box/input_files/Pk_CAMB_test.dat')

In [4]:
theory = ComputeModelsCoLoRe(
    box_path=theory_path,
    source=2,
    nz_filename=nz_filename,
    pk_filename=pk_filename,
    param_cfg_filename='/global/cscratch1/sd/damonge/CoLoRe_sims/sim1000/out_params.cfg',
    bias_filename=bias_filename,
    apply_lognormal=True)

In [5]:
class HelpGetFit:
    def __init__(self, boxes_path, zmin, zmax, rsd, rsd2, free_params=['bias'], fitter_args=dict()):
        self.boxes_path = boxes_path
        self.zmin = zmin
        self.zmax = zmax
        self.rsd = rsd
        self.rsd2 = rsd2
        self.free_params = free_params
        self.fitter_args= fitter_args

    def run_process(self):
        self.get_boxes()
        self.do_fit()
    
    def get_boxes(self):
        self.sub_boxes = FileFuncs.mix_sims(
            FileFuncs.get_full_path(self.boxes_path,
                                    rsd=self.rsd, rsd2=self.rsd2,
                                    rmin=0.1, rmax=200, N_bins=41,
                                    zmin=self.zmin, zmax=self.zmax,
                                    nside=2)
        )
        
        self.z = theory.get_zeff(zmin=self.zmin, zmax=self.zmax)
        
    def do_fit(self):
        self.fitter = Fitter(boxes=self.sub_boxes, z=self.z, theory=theory, poles=[0,2],
                        rsd=self.rsd, rsd2=self.rsd2, smooth_factor0=1, smooth_factor_rsd0=1,
                        smooth_factor_cross0=1, **self.fitter_args)
        
        _ = self.fitter.run_fit(self.free_params)
        
    def print_parameter(self, param='bias'):
        print(f'{self.fitter.out.params[param].value} +- {self.fitter.out.params[param].stderr}')


In [6]:
autocorrelation=HelpGetFit(
    f'/global/cscratch1/sd/cramirez/NBodyKit/multibias/s{source}/analysis', 
    zmin=0.5, zmax=0.7, rsd=False, rsd2=None, free_params=['bias'])

autocorrelation.run_process()

/global/homes/c/cramirez/.local/cori/3.8-anaconda-2020.11/lib/python3.8/site-packages/mcfit/mcfit.py:126: UserWarning: The default value of lowring has been changed to False, set it to True if you cannot reproduce previous results
  warnings.warn("The default value of lowring has been changed to False, "
/global/homes/c/cramirez/.local/cori/3.8-anaconda-2020.11/lib/python3.8/site-packages/mcfit/mcfit.py:217: UserWarning: The default value of extrap has been changed to False, set it to True if you cannot reproduce previous results
  warnings.warn("The default value of extrap has been changed to False, "


In [7]:
autocorrelation_rsd = HelpGetFit(
    f'/global/cscratch1/sd/cramirez/NBodyKit/multibias/s{source}/analysis',
    zmin=0.5, zmax=0.7, rsd=True, rsd2=None, free_params=['bias'])
autocorrelation_rsd.run_process()

In [8]:
cross_correlation_with_unclustered_norsd = HelpGetFit(
    f'/global/cscratch1/sd/cramirez/NBodyKit/cross_correlations/multibias/s1_s{source}_from_CoLoRe',
    zmin=0.5, zmax=0.7, rsd=True, rsd2=False, free_params=['bias', 'bias2'])
cross_correlation_with_unclustered_norsd.run_process()

In [10]:
cross_correlation_with_unclustered_rsd = HelpGetFit(
    f'/global/cscratch1/sd/cramirez/NBodyKit/cross_correlations/multibias/s1_s{source}_from_CoLoRe',
    zmin=0.5, zmax=0.7, rsd=True, rsd2=True, free_params=['bias', 'bias2'],
    fitter_args=dict(bias20=2, bias0=0.01))
cross_correlation_with_unclustered_rsd.run_process()

In [13]:
cross_correlation_with_clustered_rsd = HelpGetFit(
    f'/global/cscratch1/sd/cramirez/NBodyKit/cross_correlations/multibias/s4_s5_from_CoLoRe',
    zmin=0.5, zmax=0.7, rsd=True, rsd2=True, free_params=['bias', 'bias2'],
    fitter_args=dict(bias20=2, bias0=1))
cross_correlation_with_clustered_rsd.run_process()

In [52]:
cross_correlation_with_clustered_no_rsd = HelpGetFit(
    f'/global/cscratch1/sd/cramirez/NBodyKit/cross_correlations/multibias/s4_s5_from_CoLoRe',
    zmin=0.5, zmax=0.7, rsd=False, rsd2=False, free_params=['bias2'],
    fitter_args=dict(bias20=1.8, bias0=1.11))
cross_correlation_with_clustered_no_rsd.run_process()

In [22]:
cross_correlation_with_clustered_rsd_norsd = HelpGetFit(
    f'/global/cscratch1/sd/cramirez/NBodyKit/cross_correlations/multibias/s4_s5_from_CoLoRe',
    zmin=0.5, zmax=0.7, rsd=True, rsd2=False, free_params=['bias', 'bias2'],
    fitter_args=dict(bias20=2, bias0=1))
cross_correlation_with_clustered_rsd_norsd.run_process()

In [25]:
from tabulate import tabulate

In [55]:
header = ['correlation', 's5 bias', 'stderr']

In [56]:
cols = []
probes = (f's{source}_s{source}', f's{source}_s{source}(rsd)', f's1_s{source}', f's1_s{source}(rsd)', f's4(rsd)_s5(rsd)', f's4(norsd)_s5(norsd)', f's4(rsd)_s5(norsd)')
objects = (autocorrelation, autocorrelation_rsd, cross_correlation_with_unclustered_norsd,cross_correlation_with_unclustered_rsd, cross_correlation_with_clustered_rsd, cross_correlation_with_clustered_no_rsd, cross_correlation_with_clustered_rsd_norsd)
params = ('bias','bias','bias2','bias2', 'bias2', 'bias2', 'bias2')
for probe, obj, param in zip(probes, objects, params):
    cols.append([probe, obj.fitter.out.params[param].value, obj.fitter.out.params[param].stderr])

In [57]:
print(tabulate(cols, header, tablefmt='pretty'))

+---------------------+--------------------+-----------------------+
|     correlation     |      s5 bias       |        stderr         |
+---------------------+--------------------+-----------------------+
|        s5_s5        | 1.468077598795022  | 0.0018296503206676666 |
|     s5_s5(rsd)      | 1.4504135074013766 | 0.0019620969332990526 |
|        s1_s5        | 1.4712008528658571 | 0.004696924224144221  |
|     s1_s5(rsd)      | 1.419503159772368  |                       |
|   s4(rsd)_s5(rsd)   | 1.6234490255796303 |  0.08110871821632422  |
| s4(norsd)_s5(norsd) | 1.4953367802940658 | 0.003443502011080056  |
|  s4(rsd)_s5(norsd)  | 1.5053244733972426 |  0.01684444580090797  |
+---------------------+--------------------+-----------------------+
